In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
class Actor(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_size):
        super().__init__()
        self.fc1 = nn.Linear(state_dim, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.mu = nn.Linear(hidden_size, action_dim)
        self.log_std = nn.Linear(hidden_size, action_dim)

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        x = torch.relu(self.fc2(x))
        mu = self.mu(x)
        log_std = self.log_std(x)
        std = torch.exp(log_std)
        dist = Normal(mu, std)
        return dist

In [6]:
action = Actor(1, 1, 1)
action.parameters()

<generator object Module.parameters at 0x00000171F4B4EEA0>

In [7]:
class SACAgent(nn.Module):
    def __init__(self, observation_size, action_size, hidden_size):
        super().__init__()
        self.actor = Actor(observation_size, action_size, hidden_size)
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=.007)

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
# from mlagents_envs.environment import UnityEnvironment
# from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel
# from mlagents_envs.base_env import ActionTuple  
# import torch.distributions as D
from torch.distributions import Categorical
import numpy as np
import sys
from debug_side_channel import DebugSideChannel
from torch.distributions import Normal

TRAINING_STEPS = 10000
OBSERVATION_SIZE = 6
ACTION_SIZE = 5
HIDDEN_SIZE = 128
LEARNING_RATE = .001
UPDATE_PERIOD = 100
AGENT_ID_A = 0
AGENT_ID_B = 1
NO_AGENTS = 2
BUFFER_SIZE = 256

Unity_Environment = "C:\\Users\\rmarr\\Documents\\ml-agents-dodgeball-env-ICT"
NO_SIMULATIONS = 1
TIME_SCALE = 2.0
# state
# 10 x 10 grid 
# 0 = nothing 1 = wall 2 = agent position
#debug_side_channel = DebugSideChannel()

class Actor(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_size):
        super().__init__()
        self.fc1 = nn.Linear(state_dim, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.mu = nn.Linear(hidden_size, action_dim)
        self.log_std = nn.Linear(hidden_size, action_dim)

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        x = torch.relu(self.fc2(x))
        mu = self.mu(x)
        log_std = self.log_std(x)
        std = torch.exp(log_std)
        dist = Normal(mu, std)
        return dist



class SACAgent(nn.Module):
    def __init__(self, observation_size, action_size, hidden_size):
        super().__init__()
        self.actor = Actor(observation_size, action_size, hidden_size)

        self.target_critic1.load_state_dict(self.critic1.state_dict())
        self.target_critic2.load_state_dict(self.critic2.state_dict())

        self.critic1_optimizer = optim.Adam(self.critic1.parameters(), lr=LEARNING_RATE)
        self.critic2_optimizer = optim.Adam(self.critic2.parameters(), lr=LEARNING_RATE)
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=LEARNING_RATE)


    def q1(self, state, action):
        q_in = torch.cat((state, torch.one_hot(action)))
        x = self.fc1(q_in)
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.softmax(x, dim=0)
        return x 
        
    def q2(self, state, action):
        q_in = torch.cat((state, torch.one_hot(action)))
        x = self.fc1(q_in)
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.softmax(x, dim=0)
        return x 

    # maybe switch to paper's implementation
    def actor(self, state, no_simulations):
        x = self.fc5(state)
        x = torch.relu(x)
        x = self.fc6(x)
        x = torch.softmax(x, dim=0)
        categorical_dist = Categorical(logits=x[0])
        return categorical_dist.sample((no_simulations,))


class Driver():
    def __init__(self):
        self.agent_registry = []
        sacagent = SACAgent(observation_size=OBSERVATION_SIZE, action_size=ACTION_SIZE, hidden_size=HIDDEN_SIZE)
        

    def main(self):
        #print(f'behavior_name {behavior_name}')
        #spec = env.behavior_specs[behavior_name]
        #print(f'spec {spec}')
        #self.env.reset()
        # debug_messages = debug_side_channel.get_and_clear_messages()
        # print(debug_messages)
        # for message in debug_messages:
        #     print(f"Received from Unity: {message}")

        # TODO one optimizer or 2?

        #self.env.reset()
        for episode in range(TRAINING_STEPS):
            # Agent actions
            [self.replay_buffer_resistry[agent_id].add(self.transition_tuple(agent_id)) for agent_id in range(NO_AGENTS)]
            # done = any([self.replay_buffer_resistry[agent_id].done() for agent_id in range(NO_AGENTS)])
            # if done:
            #     self.env.reset()
            if (episode + 1) % BUFFER_SIZE == 0:
                [self.train(agent_id) for agent_id in range(NO_AGENTS)]

        self.env.close()
        

if __name__ == "__main__":
    driver = Driver()
    driver.main()


AttributeError: 'function' object has no attribute 'parameters'